In [1]:
import numpy as np
from utils import *

In [2]:
data = open('dinos.txt', 'r').read()
data = data.lower()

chars = list(set(data))
data_size, vocab_size = len(data), len(chars)

print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 19909 total characters and 27 unique characters in your data.


In [3]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }

print(char_to_ix)
print(ix_to_char)

{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [4]:
def clip(gradients, maxValue):
    '''
    Giới hạn giá trị của gradients trong khoảng giới hạn.

    Arguments:
        gradients -- một dictionary chứa các gradients 'dWaa', 'dWax', 'dWya', 'db', 'dby'
        maxValue -- mọi giá trị lớn hơn số này sẽ được đặt bằng số này, và mọi giá trị nhỏ hơn -maxValue sẽ được đặt bằng -maxValue

    Returns: 
        gradients -- một dictionary chứa các gradients đã được cắt giới hạn.
    '''
    
    dWaa, dWax, dWya, dba, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['dba'], gradients['dby']
   
    # Cắt giới hạn để giảm thiểu gradients bùng nổ, lặp qua [dWax, dWaa, dWya, db, dby].
    for gradient in [dWax, dWaa, dWya, dba, dby]:
        np.clip(gradient, -maxValue, maxValue, out=gradient)
    
    gradients = {'dWaa': dWaa, 'dWax': dWax, 'dWya': dWya, 'dba': dba, 'dby': dby}    
    return gradients

In [5]:
def sample(parameters, char_to_ix, seed):
    '''
    Tạo mẫu một chuỗi ký tự dựa trên các phân phối xác suất đầu ra của RNN

    Arguments:
        parameters -- dictionary chứa các tham số Waa, Wax, Wya, by, và b. 
        char_to_ix -- dictionary ánh xạ mỗi ký tự với một chỉ số.
        seed -- được sử dụng cho mục đích chấm điểm. 

    Returns:
        indices -- một danh sách có độ dài n chứa các chỉ số của các ký tự được lấy mẫu.
    '''
    
    # Trích xuất các tham số và dạng liên quan từ dictionary 'parameters'
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    # Bước 1: Tạo vector one-hot x cho ký tự đầu tiên (khởi tạo quá trình tạo ra chuỗi). 
    x = np.zeros((vocab_size, 1))
    
    # Bước 1': Khởi tạo a_prev là zeros
    a_prev = np.zeros((n_a, 1))
    
    # Tạo một danh sách rỗng của các chỉ số, đây là danh sách sẽ chứa danh sách các chỉ số của các ký tự cần tạo ra 
    indices = []
    
    # Idx là một cờ để phát hiện ký tự xuống dòng, chúng tôi khởi tạo nó là -1
    idx = -1 
    
    # Lặp qua các bước thời gian t. Ở mỗi bước thời gian, lấy mẫu một ký tự từ phân phối xác suất và thêm 
    # chỉ số của nó vào 'indices'. Sẽ dừng lại nếu đạt đến 50 ký tự (điều này nên rất ít xảy ra 
    # với một mô hình được đào tạo tốt), điều này giúp gỡ lỗi và ngăn chặn việc vào một vòng lặp vô hạn. 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Bước 2: Truyền tiến x qua các phương trình (1), (2) và (3)
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + ba)
        z = np.dot(Wya, a) + by
        y = softmax(z)
        
        # cho mục đích chấm điểm
        np.random.seed(counter + seed) 
        
        # Bước 3: Lấy mẫu chỉ số của một ký tự trong từ vựng từ phân phối xác suất y
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel())

        # Thêm chỉ số vào 'indices'
        indices.append(idx)
        
        # Bước 4: Ghi đè ký tự đầu vào thành ký tự tương ứng với chỉ số được lấy mẫu.
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        
        # Cập nhật 'a_prev' thành 'a'
        a_prev = a
        
        # cho mục đích chấm điểm
        seed += 1
        counter += 1

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices


In [6]:
# GRADED FUNCTION: optimize

def optimize(X, Y, a_prev, parameters, learning_rate=0.01):
    '''
    Thực hiện một bước tối ưu hóa để huấn luyện mô hình.
    
    Arguments:
        X -- danh sách các số nguyên, mỗi số nguyên là một số tương ứng với một ký tự trong từ vựng.
        Y -- danh sách các số nguyên, hoàn toàn giống như X nhưng dịch một chỉ số sang trái.
        a_prev -- trạng thái ẩn trước đó.

    Parameters -- dictionary chứa:
        Wax -- Ma trận trọng số nhân với đầu vào, mảng numpy có dạng (n_a, n_x)
        Waa -- Ma trận trọng số nhân với trạng thái ẩn, mảng numpy có dạng (n_a, n_a)
        Wya -- Ma trận trọng số liên quan đến trạng thái ẩn và đầu ra, mảng numpy có dạng (n_y, n_a)
        ba -- Sai số, mảng numpy có dạng (n_a, 1)
        by -- Sai số liên quan đến trạng thái ẩn và đầu ra, mảng numpy có dạng (n_y, 1)
        learning_rate -- tỷ lệ học của mô hình.
    
    Returns:
        loss -- giá trị của hàm mất mát (cross-entropy)
        gradients -- dictionary chứa:
            dWax -- Độ dốc của trọng số từ đầu vào đến ẩn, dạng (n_a, n_x)
            dWaa -- Độ dốc của trọng số từ ẩn đến ẩn, dạng (n_a, n_a)
            dWya -- Độ dốc của trọng số từ ẩn đến đầu ra, dạng (n_y, n_a)
            dba -- Độ dốc của vector sai số, dạng (n_a, 1)
            dby -- Độ dốc của vector sai số đầu ra, dạng (n_y, 1)
        a[len(X)-1] -- trạng thái ẩn cuối cùng, dạng (n_a, 1)
    '''
    
    # Truyền tiến qua thời gian
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # Lùi lại qua thời gian
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    # Cắt giới hạn độ dốc của bạn giữa -5 (tối thiểu) và 5 (tối đa)
    gradients = clip(gradients, 5)
    
    # Cập nhật các tham số
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    return loss, gradients, a[len(X) - 1], parameters

In [7]:
def model(data, ix_to_char, char_to_ix, vocab_size=27, dino_names=7, num_iterations=35000, n_a=50):
    '''
    Huấn luyện mô hình và tạo ra tên khủng long. 
    
    Arguments:
        data -- tập văn bản
        ix_to_char -- dictionary ánh xạ chỉ số sang ký tự
        char_to_ix -- dictionary ánh xạ ký tự sang chỉ số
        num_iterations -- số lần lặp để huấn luyện mô hình

        vocab_size -- số lượng ký tự duy nhất được tìm thấy trong văn bản, kích thước của từ vựng
        dino_names -- số lượng tên khủng long bạn muốn tạo mẫu ở mỗi lần lặp. 
        n_a -- số đơn vị của ô LSTM
    
    Returns:
        parameters -- các tham số đã học
    '''
    
    # Trích xuất n_x và n_y từ vocab_size
    n_x, n_y = vocab_size, vocab_size
    
    # Khởi tạo các tham số
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # Khởi tạo mất mát
    loss = get_initial_loss(vocab_size, dino_names)
    
    # Xây dựng danh sách tất cả các tên khủng long (ví dụ huấn luyện).
    with open('dinos.txt') as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    # Xáo trộn danh sách tất cả các tên khủng long
    np.random.seed(0)
    np.random.shuffle(examples)
    
    # Khởi tạo trạng thái ẩn của LSTM
    a_prev = np.zeros((n_a, 1))
    
    # Vòng lặp tối ưu hóa
    for j in range(num_iterations):
        
        # Sử dụng gợi ý ở trên để xác định một ví dụ huấn luyện (X,Y) 
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix['\n']]
        
        # Thực hiện một bước tối ưu hóa: Truyền tiến -> Lùi lại -> Cắt -> Cập nhật tham số
        # Chọn tỷ lệ học là 0.01
        curr_loss, gradients, a_prev, parameters = optimize(X, Y, a_prev, parameters)
        
        # Sử dụng một thủ thuật trễ để giữ cho mất mát mượt mà. Nó xảy ra ở đây để tăng tốc độ huấn luyện.
        loss = smooth(loss, curr_loss)

        # Mỗi 2000 lần lặp, tạo ra 'n' ký tự nhờ vào hàm sample() để kiểm tra xem mô hình có học tốt không
        if j % 2000 == 0:
            
            print('Iterator: %d, Loss: %f' % (j, loss) + '\n')
            
            # Số lượng tên khủng long cần in
            seed = 0
            for name in range(dino_names):
                
                # Lấy mẫu chỉ số và in chúng
                sampled_indices = sample(parameters, char_to_ix, seed)
                print_sample(sampled_indices, ix_to_char)
                
                seed += 1  # Để có kết quả giống nhau cho mục đích chấm điểm, tăng giá trị seed lên một. 
      
            print('\n')
        
    return parameters


Run the following cell, you should observe your model outputting random-looking characters at the first iteration. After a few thousand iterations, your model should learn to generate reasonable-looking names. 

In [8]:
parameters = model(data, ix_to_char, char_to_ix, vocab_size)

Iterator: 0, Loss: 23.087336

Nkzxwtdmfqoeyhsqwasjkjvu
Kneb
Kzxwtdmfqoeyhsqwasjkjvu
Neb
Zxwtdmfqoeyhsqwasjkjvu
Eb
Xwtdmfqoeyhsqwasjkjvu




Iterator: 2000, Loss: 27.884160

Liusskeomnolxeros
Hmdaairus
Hytroligoraurus
Lecalosapaus
Xusicikoraurus
Abalpsamantisaurus
Tpraneronxeros


Iterator: 4000, Loss: 25.901815

Mivrosaurus
Inee
Ivtroplisaurus
Mbaaisaurus
Wusichisaurus
Cabaselachus
Toraperlethosdarenitochusthiamamumamaon


Iterator: 6000, Loss: 24.608779

Onwusceomosaurus
Lieeaerosaurus
Lxussaurus
Oma
Xusteonosaurus
Eeahosaurus
Toreonosaurus


Iterator: 8000, Loss: 24.070350

Onxusichepriuon
Kilabersaurus
Lutrodon
Omaaerosaurus
Xutrcheps
Edaksoje
Trodiktonus


Iterator: 10000, Loss: 23.844446

Onyusaurus
Klecalosaurus
Lustodon
Ola
Xusodonia
Eeaeosaurus
Troceosaurus


Iterator: 12000, Loss: 23.291971

Onyxosaurus
Kica
Lustrepiosaurus
Olaagrraiansaurus
Yuspangosaurus
Eealosaurus
Trognesaurus


Iterator: 14000, Loss: 23.382338

Meutromodromurus
Inda
Iutroinatorsaurus
Maca
Yusteratoptititan
Ca
Troclosaurus


Iterator: 16000, Loss: 23.265759

Meustoloplohus
Imeda
Iutosaurus
Maca
Yuspanenphurus
Daaisicachtitan
Trodon


Iterator: